In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/VKR_Malceva')

Mounted at /content/drive


# Data

In [2]:
print("="*60)
print("ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ LIGHTGBM")
print("="*60)

# 1. Загрузка данных
print("1. Загрузка данных...")
train_df = pd.read_csv('./data/train_processed.csv')
val_df = pd.read_csv('./data/val_processed.csv')

# 2. Разделение на признаки и таргет
print("2. Разделение данных...")
X_train = train_df.drop(columns=['TARGET'])
y_train = train_df['TARGET']
X_val = val_df.drop(columns=['TARGET'])
y_val = val_df['TARGET']

# 3. Исправление названий столбцов для LightGBM
print("3. Исправление названий столбцов...")

def fix_column_names(df):
    """Исправляет названия столбцов для совместимости с LightGBM"""
    df_fixed = df.copy()
    new_columns = {}
    
    for col in df_fixed.columns:
        # Заменяем все не-буквенно-цифровые символы на подчеркивание
        new_name = str(col)
        # Заменяем пробелы и основные проблемные символы
        for char in [' ', '(', ')', '[', ']', '{', '}', '/', '\\', ':', ',', '.', '-', '+', '*', '&', '^', '%', '$', '#', '@', '!', '?']:
            new_name = new_name.replace(char, '_')
        
        # Убираем множественные подчеркивания
        while '__' in new_name:
            new_name = new_name.replace('__', '_')
        
        # Убираем подчеркивания в начале/конце
        new_name = new_name.strip('_')
        
        # Если имя пустое или начинается с цифры
        if not new_name:
            new_name = f'feature_{col}'
        elif new_name[0].isdigit():
            new_name = f'f_{new_name}'
        
        new_columns[col] = new_name
    
    return df_fixed.rename(columns=new_columns)

X_train = fix_column_names(X_train)
X_val = fix_column_names(X_val)

print(f"   Переименовано {X_train.shape[1]} столбцов")
print(f"   Пример: '{list(train_df.columns)[0]}' -> '{X_train.columns[0]}'")

# 4. Оптимизация типов данных
print("4. Оптимизация типов данных...")

# Целевая переменная -> int8
y_train = y_train.astype(np.int8)
y_val = y_val.astype(np.int8)

# Бинарные признаки (0/1) -> int8
binary_mask = (X_train.nunique() <= 2) & (X_train.isin([0, 1]).all())
binary_cols = X_train.columns[binary_mask].tolist()

if binary_cols:
    X_train[binary_cols] = X_train[binary_cols].astype(np.int8)
    X_val[binary_cols] = X_val[binary_cols].astype(np.int8)
    print(f"   Бинарных признаков: {len(binary_cols)} -> int8")

# Остальные признаки -> float32
other_cols = [col for col in X_train.columns if col not in binary_cols]
if other_cols:
    X_train[other_cols] = X_train[other_cols].astype(np.float32)
    X_val[other_cols] = X_val[other_cols].astype(np.float32)
    print(f"   Непрерывных признаков: {len(other_cols)} -> float32")

# 5. Удаление признаков с нулевой дисперсией
print("5. Удаление признаков с нулевой дисперсией...")
zero_var_cols = X_train.columns[X_train.std() == 0]
if len(zero_var_cols) > 0:
    X_train = X_train.drop(columns=zero_var_cols)
    X_val = X_val.drop(columns=zero_var_cols)
    print(f"   Удалено {len(zero_var_cols)} признаков с нулевой дисперсией")
else:
    print("   Признаков с нулевой дисперсией не найдено")

# 6. Очистка памяти
print("6. Очистка памяти...")
del train_df, val_df
import gc
gc.collect()

# 7. Итоговые результаты
print("\n" + "="*60)
print("ИТОГОВЫЕ РАЗМЕРЫ ДАННЫХ")
print("="*60)

print(f"X_train: {X_train.shape} (память: {X_train.memory_usage().sum()/1024**2:.1f} MB)")
print(f"X_val:   {X_val.shape} (память: {X_val.memory_usage().sum()/1024**2:.1f} MB)")
print(f"y_train: {y_train.shape} (тип: {y_train.dtype})")
print(f"y_val:   {y_val.shape} (тип: {y_val.dtype})")

print(f"\nТипы признаков в X_train:")
for dtype in X_train.dtypes.unique():
    count = (X_train.dtypes == dtype).sum()
    print(f"  {dtype}: {count} признаков")

print(f"\nПроверка пропущенных значений:")
print(f"  X_train: {X_train.isna().sum().sum()}")
print(f"  X_val:   {X_val.isna().sum().sum()}")

print(f"\nПримеры названий признаков:")
print(f"  {list(X_train.columns[:3])}")

print("\n" + "="*60)
print("ДАННЫЕ ГОТОВЫ ДЛЯ LIGHTGBM!")
print("="*60)

ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ LIGHTGBM
1. Загрузка данных...
2. Разделение данных...
3. Исправление названий столбцов...
   Переименовано 702 столбцов
   Пример: 'SK_ID_CURR' -> 'SK_ID_CURR'
4. Оптимизация типов данных...
   Бинарных признаков: 115 -> int8
   Непрерывных признаков: 587 -> float32
5. Удаление признаков с нулевой дисперсией...
   Удалено 5 признаков с нулевой дисперсией
6. Очистка памяти...

ИТОГОВЫЕ РАЗМЕРЫ ДАННЫХ
X_train: (184506, 697) (память: 432.5 MB)
X_val:   (61502, 697) (память: 144.2 MB)
y_train: (184506,) (тип: int8)
y_val:   (61502,) (тип: int8)

Типы признаков в X_train:
  float32: 587 признаков
  int8: 110 признаков

Проверка пропущенных значений:
  X_train: 0
  X_val:   0

Примеры названий признаков:
  ['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL']

ДАННЫЕ ГОТОВЫ ДЛЯ LIGHTGBM!


# LightGBM

In [5]:
print("="*80)
print("УСТАНОВКА И ИМПОРТ LIGHTGBM")
print("="*80)

# Устанавливаем LightGBM если ещё не установлен
try:
    import lightgbm as lgb
    print("✅ LightGBM уже установлен")
except:
    print("Устанавливаю LightGBM...")
    !pip install lightgbm -q
    import lightgbm as lgb
    print("✅ LightGBM установлен")


from sklearn.model_selection import RandomizedSearchCV


print(f"\nДанные: X_train={X_train.shape}, X_val={X_val.shape}")
print(f"Тип данных: {X_train.dtypes.unique()}")
print(f"Целевая переменная: {y_train.unique()}")

УСТАНОВКА И ИМПОРТ LIGHTGBM
✅ LightGBM уже установлен

Данные: X_train=(184506, 697), X_val=(61502, 697)
Тип данных: [dtype('float32') dtype('int8')]
Целевая переменная: [1 0]


In [6]:
print("="*80)
print("БЛОК 1: ПОИСК ЛУЧШИХ ГИПЕРПАРАМЕТРОВ LIGHTGBM")


# 1. Создаем базовую модель
print("\n1. Создание базовой модели LightGBM...")
base_lgb = lgb.LGBMClassifier(
    random_state=42,
    n_jobs=-1,            # Используем все ядра
    verbose=-1,           # Без вывода
    class_weight='balanced'  # Учитываем дисбаланс
)

# 2. Определяем сетку гиперпараметров для RandomizedSearchCV
print("\n2. Определение сетки гиперпараметров...")
param_distributions = {
    # Основные параметры
    'n_estimators': [100, 200, 300, 400, 500, 600, 800],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
    'num_leaves': [20, 31, 40, 50, 60, 70, 80, 100, 150],
    
    # Регуляризация
    'reg_alpha': [0, 0.01, 0.05, 0.1, 0.5, 1, 2, 5],  # L1
    'reg_lambda': [0, 0.01, 0.05, 0.1, 0.5, 1, 2, 5], # L2
    
    # Сэмплирование
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'subsample_freq': [0, 1, 5, 10],
    
    # Контроль переобучения
    'min_child_samples': [10, 20, 30, 40, 50, 100],
    'min_child_weight': [1e-5, 1e-3, 0.01, 0.1, 1],
    'min_split_gain': [0, 0.001, 0.01, 0.1],
    
    # Другие
    'max_depth': [-1, 5, 10, 15, 20, 25],  # -1 = без ограничений
    'boosting_type': ['gbdt', 'dart', 'goss'],  # Тип бустинга
    'importance_type': ['split', 'gain']  # Тип важности признаков
}

print(f"   Всего параметров для поиска: {sum(len(v) if isinstance(v, list) else 1 for v in param_distributions.values())}")

# 3. Настройка RandomizedSearchCV
print("\n3. Настройка RandomizedSearchCV...")
random_search = RandomizedSearchCV(
    estimator=base_lgb,
    param_distributions=param_distributions,
    n_iter=30,           # 30 случайных комбинаций
    cv=3,                # 3 фолда кросс-валидации
    scoring='roc_auc',   # Метрика для оптимизации
    n_jobs=1,            # LightGBM уже параллелит, тут ставим 1
    verbose=10,          # Детальный вывод прогресса
    random_state=42,
    refit=False,         # Пока не обучаем финальную модель
    return_train_score=True
)

# 4. Запуск поиска гиперпараметров
print("\n4. Запуск поиска лучших гиперпараметров...")
print(f"   Будет проверено {30 * 3} моделей (30 итераций × 3 фолда)")
print(f"   Размер данных: {X_train.shape}")
print(f"   Оценка времени: ~{30 * 3 * 0.5 / 60:.1f}-{30 * 3 * 2 / 60:.1f} минут (LightGBM очень быстрая!)")
print("-"*80)

import time
start_time = time.time()
random_search.fit(X_train, y_train)
search_time = time.time() - start_time

# 5. Результаты поиска
print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ ПОИСКА ГИПЕРПАРАМЕТРОВ LIGHTGBM")
print("="*80)

print(f"Поиск завершен за {search_time/60:.1f} минут")
print(f"\nЛучшие параметры:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nЛучший ROC-AUC на кросс-валидации: {random_search.best_score_:.4f}")

# 6. Показываем топ-5 комбинаций параметров
print("\nТоп-5 лучших комбинаций параметров:")
results_df = pd.DataFrame(random_search.cv_results_)
top_results = results_df.sort_values('mean_test_score', ascending=False).head(5)

for i, (_, row) in enumerate(top_results.iterrows(), 1):
    params = row['params']
    print(f"\n{i}. Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})")
    print(f"   n_estimators={params.get('n_estimators')}, learning_rate={params.get('learning_rate')}, "
          f"num_leaves={params.get('num_leaves')}")

print("\n" + "="*80)
print("СОХРАНЕНИЕ ЛУЧШИХ ПАРАМЕТРОВ ДЛЯ ЛОКАЛЬНОГО ОБУЧЕНИЯ")
print("="*80)

# 7. Сохраняем лучшие параметры в файл
best_params = random_search.best_params_

# Конвертируем типы для безопасности (LightGBM может быть чувствителен)
clean_best_params = {}
for key, value in best_params.items():
    # Преобразуем numpy типы в стандартные Python типы
    if isinstance(value, (np.integer, np.floating)):
        clean_best_params[key] = value.item()
    else:
        clean_best_params[key] = value

import json
import joblib

# Сохраняем в JSON (удобно для переноса)
with open('./models/lgb_best_params.json', 'w') as f:
    json.dump(clean_best_params, f, indent=2)

# Сохраняем весь объект поиска
search_results = {
    'best_params': clean_best_params,
    'best_score': random_search.best_score_,
    'cv_results': results_df.to_dict('records'),
    'search_time_minutes': search_time / 60
}

joblib.dump(search_results, './models/lgb_search_results.pkl')

print(f"\n✅ Лучшие параметры сохранены:")
print(f"   JSON: ./models/lgb_best_params.json")
print(f"   Полные результаты: ./models/lgb_search_results.pkl")
print(f"\nСкопируй эти параметры для обучения на своём компьютере!")

БЛОК 1: ПОИСК ЛУЧШИХ ГИПЕРПАРАМЕТРОВ LIGHTGBM

1. Создание базовой модели LightGBM...

2. Определение сетки гиперпараметров...
   Всего параметров для поиска: 78

3. Настройка RandomizedSearchCV...

4. Запуск поиска лучших гиперпараметров...
   Будет проверено 90 моделей (30 итераций × 3 фолда)
   Размер данных: (184506, 697)
   Оценка времени: ~0.8-3.0 минут (LightGBM очень быстрая!)
--------------------------------------------------------------------------------
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV 1/3; 1/30] START boosting_type=gbdt, colsample_bytree=0.7, importance_type=split, learning_rate=0.05, max_depth=15, min_child_samples=30, min_child_weight=0.01, min_split_gain=0.001, n_estimators=800, num_leaves=60, reg_alpha=0, reg_lambda=0, subsample=0.9, subsample_freq=5
[CV 1/3; 1/30] END boosting_type=gbdt, colsample_bytree=0.7, importance_type=split, learning_rate=0.05, max_depth=15, min_child_samples=30, min_child_weight=0.01, min_split_gain=0.001, n_esti

In [3]:
print("="*80)
print("БЛОК 2: ОБУЧЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ LIGHTGBM С ЛУЧШИМИ ПАРАМЕТРАМИ")
print("="*80)

import json
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score
import lightgbm as lgb

# 1. Загрузка лучших параметров
print("\n1. Загрузка лучших параметров...")
try:
    with open('./models/lgb_best_params.json', 'r') as f:
        best_params = json.load(f)
    print(f"✅ Загружено {len(best_params)} параметров")
except FileNotFoundError:
    print("❌ Файл с параметрами не найден!")
    print("Использую лучшие параметры из random_search...")
    best_params = clean_best_params  # из предыдущего кода

# 2. Создание финальной модели
print("\n2. Создание финальной модели LightGBM...")
final_lgb = lgb.LGBMClassifier(
    **best_params,
    random_state=42,
    n_jobs=-1,            # Используем все ядра
    class_weight='balanced'
)

print(f"Параметры модели:")
for key, value in list(best_params.items())[:10]:  # покажем первые 10
    print(f"  {key}: {value}")
if len(best_params) > 10:
    print(f"  ... и еще {len(best_params) - 10} параметров")

# 3. Обучение на всех тренировочных данных
print(f"\n3. Обучение на всех {len(X_train)} samples...")
print(f"   Размер данных: {X_train.shape}")
print(f"   Количество признаков: {X_train.shape[1]}")

import time
train_start = time.time()
final_lgb.fit(X_train, y_train)
train_time = time.time() - train_start

print(f"\n✅ Модель обучена за {train_time/60:.1f} минут")
print(f"   Количество деревьев: {final_lgb.n_estimators_}")
print(f"   Количество итераций: {final_lgb.n_iter_}")

# 4. Предсказания на валидационной выборке
print(f"\n4. Предсказания на валидации ({len(X_val)} samples)...")
y_val_pred = final_lgb.predict(X_val)
y_val_pred_proba = final_lgb.predict_proba(X_val)[:, 1]

# 5. Расчет метрик
print("\n5. Расчет метрик...")
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred, average='macro')
roc_auc = roc_auc_score(y_val, y_val_pred_proba)

# Для полноты можно добавить precision и recall
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
recall = recall_score(y_val, y_val_pred, average='macro', zero_division=0)

# 6. Вывод результатов
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ LIGHTGBM НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ")
print("="*60)

print(f"\nОсновные метрики:")
print(f"  Accuracy:   {accuracy:.4f}")
print(f"  Precision:  {precision:.4f}")
print(f"  Recall:     {recall:.4f}")
print(f"  F1 Macro:   {f1:.4f}")
print(f"  ROC-AUC:    {roc_auc:.4f}")

# 7. Classification Report
print("\n" + "-"*60)
print("Classification Report:")
print("-"*60)
print(classification_report(y_val, y_val_pred, zero_division=0))

# 8. Confusion Matrix
print("\n" + "-"*60)
print("Confusion Matrix:")
print("-"*60)
cm = confusion_matrix(y_val, y_val_pred)
print(cm)

# 9. Важность признаков
print("\n" + "="*60)
print("ВАЖНОСТЬ ПРИЗНАКОВ LIGHTGBM (TOP-30)")
print("="*60)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_lgb.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nСамые важные признаки:")
print(feature_importance.head(30).to_string(index=False))

print(f"\nСтатистика важности признаков:")
print(f"  Всего признаков: {len(feature_importance)}")
print(f"  Признаков с importance > 0: {(feature_importance['importance'] > 0).sum()}")
print(f"  Максимальная importance: {feature_importance['importance'].max():.4f}")
print(f"  Минимальная importance: {feature_importance['importance'].min():.4f}")
print(f"  Средняя importance: {feature_importance['importance'].mean():.6f}")

# 10. Сохранение финальной модели и результатов
print("\n" + "="*60)
print("СОХРАНЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ И РЕЗУЛЬТАТОВ")
print("="*60)

# Подготовка данных для сохранения
lgb_final_artifacts = {
    'model': final_lgb,
    'best_params': best_params,
    'cv_score': random_search.best_score_ if 'random_search' in locals() else None,
    'val_metrics': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_macro': f1,
        'roc_auc': roc_auc,
        'confusion_matrix': cm.tolist()
    },
    'feature_importance': feature_importance.to_dict('records'),
    'training_info': {
        'model_type': 'LGBMClassifier',
        'training_time_minutes': train_time / 60,
        'n_estimators': final_lgb.n_estimators_,
        'n_iterations': final_lgb.n_iter_,
        'feature_count': X_train.shape[1],
        'training_samples': len(X_train),
        'validation_samples': len(X_val),
        'boosting_type': best_params.get('boosting_type', 'gbdt'),
        'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}

# Сохраняем модель
joblib.dump(lgb_final_artifacts, './models/lgb_final_model.pkl')
print("✅ Финальная модель сохранена: './models/lgb_final_model.pkl'")


# 11. Итоговая информация
print("\n" + "="*80)
print("ИТОГОВАЯ ИНФОРМАЦИЯ О LIGHTGBM МОДЕЛИ")
print("="*80)

print(f"\nМодель: LightGBM Classifier")
if 'random_search' in locals():
    print(f"Лучший CV ROC-AUC: {random_search.best_score_:.4f}")
print(f"Валидационный ROC-AUC: {roc_auc:.4f}")
print(f"Accuracy на валидации: {accuracy:.4f}")
print(f"F1 Macro на валидации: {f1:.4f}")

print(f"\nХарактеристики модели:")
print(f"  Количество деревьев: {final_lgb.n_estimators_}")
print(f"  Тип бустинга: {best_params.get('boosting_type', 'gbdt')}")
print(f"  Learning rate: {best_params.get('learning_rate', 0.1)}")
print(f"  Количество листьев: {best_params.get('num_leaves', 31)}")
print(f"  Важных признаков (importance > 0): {(feature_importance['importance'] > 0).sum()}")

print(f"\nВремя обучения: {train_time/60:.1f} минут")

print("\n" + "="*80)
print("LIGHTGBM МОДЕЛЬ ОБУЧЕНА И ОЦЕНЕНА!")
print("="*80)

БЛОК 2: ОБУЧЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ LIGHTGBM С ЛУЧШИМИ ПАРАМЕТРАМИ

1. Загрузка лучших параметров...
✅ Загружено 14 параметров

2. Создание финальной модели LightGBM...
Параметры модели:
  subsample_freq: 1
  subsample: 0.9
  reg_lambda: 0.5
  reg_alpha: 5
  num_leaves: 60
  n_estimators: 600
  min_split_gain: 0.01
  min_child_weight: 0.001
  min_child_samples: 10
  max_depth: 20
  ... и еще 4 параметров

3. Обучение на всех 184506 samples...
   Размер данных: (184506, 697)
   Количество признаков: 697
[LightGBM] [Info] Number of positive: 14895, number of negative: 169611
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.206676 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 62940
[LightGBM] [Info] Number of data points in the train set: 184506, number of used features: 668
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000

In [4]:
# 1. В начале ноутбука создаем пустой список
model_results = []

# 2. После КАЖДОЙ обученной модели:
def log_model(model, X_val, y_val, name):
    """Записывает метрики модели в таблицу"""
    
    y_pred = model.predict(X_val)
    
    result = {
        'model': name,
        'accuracy': round(accuracy_score(y_val, y_pred), 4),
        'f1': round(f1_score(y_val, y_pred, average='macro'), 4)
    }
    
    # ROC-AUC если доступен
    if hasattr(model, 'predict_proba'):
        try:
            y_proba = model.predict_proba(X_val)
            if len(model.classes_) == 2:
                result['roc_auc'] = round(roc_auc_score(y_val, y_proba[:, 1]), 4)
        except:
            pass
    
    model_results.append(result)
    
    # Создаем DataFrame и показываем
    results_df = pd.concat([pd.read_csv('results.csv'), pd.DataFrame(model_results)], ignore_index=True)
    print(f"\nДобавлена модель: {name}")
    print(results_df.to_string(index=False))
    
    return results_df

In [5]:

artifacts = joblib.load('./models/lgb_final_model.pkl')

results = log_model(artifacts['model'], X_val, y_val, 'LightGBM')
results.to_csv('results.csv', index=False)


Добавлена модель: LightGBM
                 model  accuracy     f1  roc_auc
         SGDClassifier    0.7016 0.5416   0.7617
 DcisionTreeClassifier    0.6481 0.5034   0.7082
RandomForestClassifier    0.9171 0.5336   0.7524
              LightGBM    0.7645 0.5776   0.7733
